In [61]:
from __future__ import division
import psycopg2
import collections
import json

In [62]:
def load_table(json_file, conn):
    cur = conn.cursor()
    cur.execute("SET TIME ZONE 'UTC';")

    data_path = '/Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/'
    json_path = data_path + json_file + '.json'
    print 'processing %s...' % json_path
    
    with open(json_path,'r') as f:
        # data = bson.json_util.loads(f.read())
        data = json.JSONDecoder(object_pairs_hook=collections.OrderedDict).decode(f.read())
        query = tables['reports']
        for key, val in data[json_file].iteritems():
            if type(val) == collections.OrderedDict:
                cur.execute(query, val.values() + [int(key)]) 
            else: #assuming list, try to explode
                for x in val:
                    cur.execute(query, x.values() + [int(key)])
    conn.commit()

In [63]:
tables = {
    'assigned_to': '',
    'bug_status': '',
    'cc': '',
    'component': '',
    'op_sys': '',
    'priority': '',
    'product': '',
    'reports': 'INSERT INTO reports VALUES (to_timestamp(%s), %s, %s, %s, %s)',
    'resolution': '',
    'severity': '',
    'short_desc': '',
    'version': '',
}

In [65]:
try:
    conn = psycopg2.connect("dbname='bugs' user='lucka' host='localhost'")
except:
    print "Unable to connect to the database"
    
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS reports (
    opening             timestamp,
    reporter            bigint,
    current_status      varchar(50),
    current_resolution  varchar(50),
    id                  bigint NOT NULL
    );
""")
conn.commit()

load_table('reports', conn)

conn.close()

processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/reports.json...


KeyboardInterrupt: 